# 2.1 DNA分词器构建

## **分词算法**

### **什么是分词**
分词就是把一个文本序列，分成一个一个的token/词，对于英文这种天生带空格的语言，一般使用空格和标点分词就行了，而对于中文等语言，并没有特殊的符号来分词，因此，一般需要设计专门的分词算法，对于大模型而言，一般需要处理多种语言，因此，也需要专门的分词算法。

在大模型（如 BERT、GPT 系列、T5 等）中，分词器（tokenizer）扮演着至关重要的角色。它负责将原始文本转换为模型可以处理的格式，即将文本分解成 token 序列，并将这些 token 映射到模型词汇表中的唯一 ID。分词器的选择和配置直接影响模型的性能和效果。以下是几种常见的分词器及其特点，特别关注它们在大型语言模型中的应用。

### 1. **WordPiece 分词器**

- **使用场景**：广泛应用于 BERT 及其变体。
- **工作原理**：基于频率统计，从语料库中学习最有效的词汇表。它根据子词（subword）在文本中的出现频率来决定如何分割单词。例如，“playing” 可能被分为 “play” 和 “##ing”，其中“##”表示该部分是前一个 token 的延续。
- **优点**：
  - 处理未知词汇能力强，能够将未见过的词汇分解为已知的子词。
  - 兼容性好，适合多种语言任务。
- **缺点**：
  - 需要额外的标记（如 `##`）来指示子词，可能影响某些应用场景下的可读性。

### 2. **Byte Pair Encoding (BPE)**

- **使用场景**：广泛应用于 GPT 系列、RoBERTa、XLM-R 等模型。
- **工作原理**：通过迭代地合并最常见的字符对来构建词汇表。BPE 是一种无监督的学习方法，能够在不依赖于预先定义的词汇表的情况下进行分词。
- **优点**：
  - 灵活性高，适应性强，尤其适用于多语言模型。
  - 不需要特殊标记，生成的词汇表更简洁。
- **缺点**：
  - 对于某些语言或领域特定的词汇，可能会产生较短的子词，导致信息丢失。

### 3. **SentencePiece**

- **使用场景**：常见于 T5、mBART 等多语言模型。
- **工作原理**：结合了 BPE 和 WordPiece 的优点，同时支持字符级和词汇级分词。它可以在没有空格的语言（如中文、日文）中表现良好。
- **优点**：
  - 支持无空格语言，适合多语言处理。
  - 学习速度快，适应性强。
- **缺点**：
  - 对于某些特定领域的专业术语，可能需要额外的预处理步骤。

### 4. **Character-Level Tokenizer**

- **使用场景**：较少用于大型语言模型，但在某些特定任务（如拼写检查、手写识别）中有应用。
- **工作原理**：直接将每个字符视为一个 token。这种方式简单直接，但通常会导致较大的词汇表。
- **优点**：
  - 简单易实现，不需要复杂的训练过程。
  - 对于字符级别的任务非常有效。
- **缺点**：
  - 词汇表较大，计算资源消耗较多。
  - 捕捉上下文信息的能力较弱。

### 5. **Unigram Language Model**

- **使用场景**：主要用于 SentencePiece 中。
- **工作原理**：基于概率分布，选择最优的分词方案以最大化似然函数。这种方法类似于 BPE，但在构建词汇表时考虑了更多的统计信息。
- **优点**：
  - 统计基础强，优化效果好。
  - 适应性强，适用于多种语言和任务。
- **缺点**：
  - 计算复杂度较高，训练时间较长。

### 分词器的关键特性

无论选择哪种分词器，以下几个关键特性都是设计和应用中需要考虑的：

- **词汇表大小**：决定了模型所能识别的词汇量。较大的词汇表可以捕捉更多细节，但也增加了内存和计算需求。
- **处理未知词汇的能力**：好的分词器应该能够有效地处理未登录词（OOV, Out-Of-Vocabulary），将其分解为已知的子词。
- **多语言支持**：对于多语言模型，分词器应能处理不同语言的文本，尤其是那些没有明显分隔符的语言。
- **效率和速度**：分词器的执行速度直接影响整个数据处理管道的效率，尤其是在大规模数据集上。
- **兼容性和灵活性**：分词器应与目标模型架构兼容，并且能够灵活适应不同的任务需求。

## DNA等生物序列分词
在生物信息学中，DNA 和蛋白质序列的处理与自然语言处理（NLP）有相似之处，但也有其独特性。为了提取这些生物序列的特征并用于机器学习或深度学习模型，通常需要将长序列分解成更小的片段（类似于 NLP 中的“分词”），以便更好地捕捉局部和全局特征。以下是几种常见的方法，用于对 DNA 和蛋白质序列进行“分词”，以提取有用的特征。

### 1. **K-mer 分解**

**定义**：K-mer 是指长度为 k 的连续子序列。例如，在 DNA 序列中，一个 3-mer 可能是 "ATG" 或 "CGA"。

**应用**：
- **DNA 序列**：常用的 k 值范围从 3 到 6。较小的 k 值可以捕捉到更细粒度的信息，而较大的 k 值则有助于识别更长的模式。
- **蛋白质序列**：k 值通常较大，因为氨基酸的数量较多（20 种），较长的 k-mer 可以捕捉到重要的结构域或功能区域。

**优点**：
- 简单且直观，易于实现。
- 可以捕捉到短序列中的局部特征。

**缺点**：
- 对于非常长的序列，生成的 k-mer 数量会非常大，导致维度爆炸问题。
- 不同位置的 k-mer 之间缺乏上下文关系。

In [2]:
#示例代码（Python）

def k_mer(seq, k):
    return [seq[i:i+k] for i in range(len(seq) - k + 1)]

dna_sequence = "ATGCGTACGTA"
protein_sequence = "MKQHKAMIVALIVLITAY"

print("DNA 3-mers:", k_mer(dna_sequence, 3))
print("Protein 4-mers:", k_mer(protein_sequence, 4))

DNA 3-mers: ['ATG', 'TGC', 'GCG', 'CGT', 'GTA', 'TAC', 'ACG', 'CGT', 'GTA']
Protein 4-mers: ['MKQH', 'KQHK', 'QHKA', 'HKAM', 'KAMI', 'AMIV', 'MIVA', 'IVAL', 'VALI', 'ALIV', 'LIVL', 'IVLI', 'VLIT', 'LITA', 'ITAY']


### 2. **滑动窗口**

**定义**：滑动窗口方法通过设定一个固定大小的窗口沿着序列移动，并在每个位置提取窗口内的子序列。这与 K-mer 类似，但允许重叠。

**应用**：
- **DNA 和蛋白质序列**：窗口大小可以根据具体任务调整，如基因预测、蛋白质结构预测等。

**优点**：
- 提供了更多的灵活性，可以控制窗口的步长和大小。
- 有助于捕捉局部和全局特征。

**缺点**：
- 计算复杂度较高，尤其是当窗口大小较大时。

In [4]:
def sliding_window(seq, window_size, step=1):
    return [seq[i:i+window_size] for i in range(0, len(seq) - window_size + 1, step)]

dna_sequence = "ATGCGTACGTA"
protein_sequence = "MKQHKAMIVALIVLITAY"

print("Sliding window (DNA, size=3, step=1):", sliding_window(dna_sequence, 3))
print("Sliding window (Protein, size=4, step=2):", sliding_window(protein_sequence, 4, step=2))

Sliding window (DNA, size=3, step=1): ['ATG', 'TGC', 'GCG', 'CGT', 'GTA', 'TAC', 'ACG', 'CGT', 'GTA']
Sliding window (Protein, size=4, step=2): ['MKQH', 'QHKA', 'KAMI', 'MIVA', 'VALI', 'LIVL', 'VLIT', 'ITAY']


### 3. **词表分词和嵌入式表示**

**定义**：使用预训练的嵌入模型（如 Word2Vec、BERT 等）来将每个 token 映射到高维向量空间中。对于生物序列，可以使用专门设计的嵌入模型，如 ProtTrans、ESM 等。

**应用**：
- **DNA 和蛋白质序列**：嵌入模型可以捕捉到序列中的语义信息和上下文依赖关系。

**优点**：
- 捕捉到丰富的语义信息，适合复杂的下游任务。
- 可以利用大规模预训练模型的优势。

**缺点**：
- 需要大量的计算资源来进行预训练。
- 模型复杂度较高，解释性较差。

In [5]:
import subprocess
import os
# 设置环境变量, autodl一般区域
result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

"""
import os

# 设置环境变量, autodl专区 其他idc
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

# 打印环境变量以确认设置成功
print(os.environ.get('HF_ENDPOINT'))
"""

In [15]:
from transformers import AutoTokenizer, AutoModel
import torch

# 加载预训练的蛋白质嵌入模型
tokenizer = AutoTokenizer.from_pretrained("dnagpt/gpt_dna_v0")
model = AutoModel.from_pretrained("dnagpt/gpt_dna_v0")

dna_sequence = "ATGCGTACGTA"
print(tokenizer.tokenize(dna_sequence))

# 编码序列
inputs = tokenizer(dna_sequence, return_tensors="pt")

# 获取嵌入
with torch.no_grad():
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state

print("Embeddings shape:", embeddings.shape)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


['ATGCG', 'TACG', 'T', 'A']
Embeddings shape: torch.Size([1, 4, 768])


### **训练DNA BPE分词器**

以上方法展示了如何对 DNA 和蛋白质序列进行“分词”，以提取有用的特征。选择哪种方法取决于具体的任务需求和数据特性。对于简单的分类或回归任务，K-mer 分解或滑动窗口可能是足够的；而对于更复杂的任务，如序列标注或结构预测，基于词汇表的方法或嵌入表示可能会提供更好的性能。

目前大部分生物序列大模型的论文中，使用最多的依然是传统的K-mer，但一些SOTA的论文则以BEP为主。而BEP分词也是目前GPT、llama等主流自然语言大模型使用的基础分词器。

因此，我们也演示下从头训练一个DNA BPE分词器的方法。

我们首先看下GPT2模型，默认的分词器，对DNA序列分词的结果：

In [10]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)
from transformers import AutoTokenizer

In [15]:
gpt2_tokenizer = AutoTokenizer.from_pretrained('gpt2')
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token

In [16]:
gpt2_tokenizer.tokenize("TGGCGTGAACCCGGGATCGGG")

['T', 'GG', 'C', 'GT', 'GA', 'AC', 'CC', 'GG', 'G', 'AT', 'C', 'GG', 'G']

可以看到，gpt2模型因为是以英文为主的BPE分词模型，分解的都是1到2个字母的结果，这样显然很难充分表达生物语义，因此，我们使用DNA序列来训练1个BPE分词器，代码也非常简单：

In [2]:
tokenizer = Tokenizer(models.BPE())
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False, use_regex=False) #use_regex=False,空格当成一般字符串
trainer = trainers.BpeTrainer(vocab_size=30000, special_tokens=["<|endoftext|>"]) #3w words

In [3]:
tokenizer.train(["../01-data_env/data/dna_1g.txt"], trainer=trainer) #all file list, take 10-20 min

In [4]:
encoding = tokenizer.encode("TGGCGTGAACCCGGGATCGGG")
print(encoding.tokens)

['TG', 'GCGTGAA', 'CCCGG', 'GATCGG', 'G']


可以看到，以DNA数据训练的分词器，分词效果明显要好的多，各种长度的词都有。

In [5]:
tokenizer.save("dna_bpe_dict.json")

In [6]:
#然后我们可以使用from_file() 方法从该文件里重新加载 Tokenizer 对象：
new_tokenizer = Tokenizer.from_file("dna_bpe_dict.json")

In [7]:
#要在 🤗 Transformers 中使用这个标记器，我们必须将它包裹在一个 PreTrainedTokenizerFast 类中
from transformers import GPT2TokenizerFast
dna_tokenizer = GPT2TokenizerFast(tokenizer_object=new_tokenizer)
dna_tokenizer.save_pretrained("dna_bpe_dict")
#dna_tokenizer.push_to_hub("dna_bpe_dict_1g", organization="dnagpt", use_auth_token="hf_*****") # push to huggingface

('dna_bpe_dict/tokenizer_config.json',
 'dna_bpe_dict/special_tokens_map.json',
 'dna_bpe_dict/vocab.json',
 'dna_bpe_dict/merges.txt',
 'dna_bpe_dict/added_tokens.json',
 'dna_bpe_dict/tokenizer.json')

In [11]:
tokenizer_new = AutoTokenizer.from_pretrained('dna_bpe_dict')

In [12]:
tokenizer_new.tokenize("TGGCGTGAACCCGGGATCGGG")

['TG', 'GCGTGAA', 'CCCGG', 'GATCGG', 'G']